In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

**Mathematical expression of the algorithm :**
<br>
For one example $x(i):$
<br>
<br>
<center>
$ z^{(i)} = w^Tx^{(i)} + b$
    <br>
$ \hat{y} = a^{(i)} = \sigma (z^{(i)}) $ 
    <br>
$ $    
    
    
</center    


**The Sigmoid function :**
<center>
    $ \sigma (z)$ = ${1}\over{1 + e^{-z} } $
</center>
<br>


In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    """
    return 1/(1+ np.exp(-z))
    

In [ ]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    w = np.zeros((dim,1))
    b = 0
   
    return w,b

#### Forward and Backward propagation

In [ ]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    Z = np.dot(w.T,X) + b
    A = sigmoid(Z)
    cost = -1/m*(np.sum(Y*np.log(A) + (1-Y)*np.log(1-A)))

    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = 1/m*(np.dot(X,(A-Y).T))
    db = 1/m*(np.sum(A-Y))
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost    
    
    
    
    

#### Optimization

In [ ]:
def optimize(w, b, X, Y, epoch, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    costs = []
    
    for i in range(epoch):
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads
        dw = grads['dw']
        db = grads['db']
        
        #update parameters
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs
            
        costs.append(cost)
        
        # Print the cost
        if print_cost:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    
    return params, grads, costs        
        
    
    

In [ ]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)    
    
    A = sigmoid(np.dot(w.T, X)+ b)
    
    for i in range(A.shape[1]):
        Y_prediction[0][i] = 1 if A[0][i] >= 0.5 else 0
        
    assert(Y_prediction.shape == (1, m))
    return Y_prediction

#### Model

In [ ]:
def model(X_train, Y_train, X_test, Y_test, epoch = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])    
    
    parameters, grads, costs = optimize(w, b, X_train, Y_train, epoch, learning_rate, print_cost = False)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "epoch": epoch}
    
    return d    
    

#### Import the necessary dataset and libraries

In [ ]:
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error,mean_squared_error,accuracy_score
from math import sqrt
from sklearn import preprocessing
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
df.head()

# Exploratory Data Analysis

Let's begin some exploratory data analysis! We'll start by checking out missing data!

#### Missing values
Let's deal with the missing values first!

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='inferno')

In [ ]:
#deleting 'Unnamed: 32' & 'id' columns
del df['Unnamed: 32']
del df['id']

In [ ]:
#get a general overview of the columns
df.describe().T

In [ ]:
sns.countplot(data = df,x = 'diagnosis',label = "Count")

In [ ]:
#feature engineering
df['diagnosis'] = df['diagnosis'].apply(lambda x : 1 if x == 'B' else 0 )

### Model Preparation

In [ ]:
y = df['diagnosis']
X = df.drop('diagnosis', axis=1)

In [ ]:
#split the train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.27, random_state=52)

In [ ]:
#fix the damn dimnesions
X_train = X_train.T
X_test = X_test.T

In [ ]:
Y_train = np.array(Y_train).reshape(1,X_train.shape[1])
Y_test = np.array(Y_test).reshape(1,X_test.shape[1])

### Training the model (simple logistic regression)

In [ ]:
d = model(X_train, Y_train, X_test, Y_test, epoch = 2000, learning_rate = 0.05, print_cost = False)

#### Testing different learning rates

In [ ]:
learning_rates = [0.1,0.01,0.001,0.005,0.0005,0.000001]
for rate in learning_rates:
    print(f'learning rate -- {rate}')
    model(X_train, Y_train, X_test, Y_test, epoch = 9999, learning_rate = rate, print_cost = False)
    print('---------')